In [42]:
import numpy as np
import matplotlib.pyplot as plt
import os
os.chdir("/home/zongchen/F2BMLD")
import matplotlib.cm as cm
import matplotlib
import sys
sys.path.append("/home/zongchen/F2BMLD")
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Sequence, Tuple
from functools import partial
from tqdm import tqdm

plt.rcParams['axes.grid'] = True
plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']
plt.rcParams['axes.labelsize'] = 18
plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r'\usepackage{amsmath, amsfonts}')

# plt.rc('font', family='Arial', size=12)
plt.rc('axes', titlesize=16, labelsize=18, grid=True)
plt.rc('lines', linewidth=2)
plt.rc('legend', fontsize=18, frameon=False)
plt.rc('xtick', labelsize=14, direction='in')
plt.rc('ytick', labelsize=14, direction='in')
plt.rc('figure', figsize=(6, 4), dpi=100)

In [43]:
import gymnasium as gym
from methods import dfiv, f2bmld
import utils
from utils import load_pretrained_dqn
env = gym.make("CartPole-v1")

env_spec = {
        "obs_dim": 4,
        "act_dim": 2,
        "obs_space": env.observation_space,
        "act_space": env.action_space,
    }

value_layer_sizes = "50"
instrumental_layer_sizes= "50"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

value_func, instrumental_feature = dfiv.bsuite_network.make_value_func_bsuite(
    env_spec, value_layer_sizes, instrumental_layer_sizes, device
)

policy_dqn = load_pretrained_dqn("policy_net.pth", device)

treatment_net = f2bmld.bsuite_network.TreatmentNetwork(env_spec, layer_sizes=[50, 1]).to(device)

def target_policy(obs_batch: torch.Tensor, policy_dqn: torch.nn.Module) -> torch.Tensor:
    with torch.no_grad():
        return policy_dqn(obs_batch).argmax(dim=-1)

class RandomActionWrapper(gym.Wrapper):
    """With probability `eps`, take a random action instead of the policy's action."""
    def __init__(self, env, eps=0.1):
        super().__init__(env)
        self.eps = eps

    def step(self, action):
        if np.random.rand() < self.eps:
            action = self.env.action_space.sample()
        return self.env.step(action)

In [44]:
def eval(environment, policy, value_func, num_init_samples, device):
    q0s = []
    for _ in tqdm(range(num_init_samples)):
        obs, _ = environment.reset()
        obs_tensor = torch.tensor(obs, dtype=torch.float32, device=device).unsqueeze(0)
        action = policy(obs_tensor)
        q0 = value_func(obs_tensor, action).item()
        q0s.append(q0)

    q0s = np.array(q0s)
    return q0s.mean()


def estimate_true_value(policy, environment, discount=0.99, 
                        num_episodes=100, device="cpu"):
    """
    Estimate the true discounted return of a policy by Monte Carlo rollouts.

    Args:
        policy: Callable/nn.Module mapping obs -> action (PyTorch).
        environment: Gym-like env with reset() and step().
        discount: float, discount factor.
        num_episodes: int, number of episodes to average over.
        device: str, "cpu" or "cuda".

    Returns:
        mean_return: average discounted return across episodes.
        stderr_return: standard error of returns.
    """
    returns = []
    for _ in tqdm(range(num_episodes)):
        obs, _ = environment.reset()
        done, ep_return, t = False, 0.0, 0

        while not done:
            obs_tensor = torch.tensor(obs, dtype=torch.float32, device=device).unsqueeze(0)
            action = policy(obs_tensor)

            # handle discrete vs continuous
            if isinstance(action, torch.Tensor):
                if action.numel() == 1:  # discrete scalar
                    action = int(action.item())
                else:  # e.g. policy outputs logits/probs
                    action = int(torch.argmax(action, dim=-1).item())

            next_obs, reward, terminated, truncated, _ = environment.step(action)
            done = terminated or truncated
            ep_return += (discount ** t) * reward
            obs = next_obs
            t += 1

        returns.append(ep_return)

    mean_return = float(np.mean(returns))
    return mean_return, float(np.std(returns)) / np.sqrt(num_episodes)

In [48]:
seed_list = [0]
env_noise_list = [0.1]
policy_noise_list = [0.0, 0.2]
# reg = "0.0001"
reg = "1e-05"

for seed in seed_list:
    for env_noise in env_noise_list:
        for policy_noise in policy_noise_list:
            ### F2BMLD
            path = f"/home/zongchen/F2BMLD/results/f2bmld_env_noise_{env_noise}__policy_noise_{policy_noise}"
            path += f"__lagrange_0.3__stage1_reg_{reg}__stage2_reg_{reg}__treat_lr_0.001"
            path += f"__instr_lr_0.001__instr_iter_10__instr_tilde_iter_10__bs_32__seed_{seed}/treatment_net.pth"
            treatment_net.load_state_dict(torch.load(path, map_location=device))

            if env_noise > 0.0:
                environment = RandomActionWrapper(env, eps=env_noise)
            else:
                environment = env
            truth_value, truth_value_ste = estimate_true_value(
                partial(target_policy, policy_dqn=policy_dqn), environment, discount=0.99, num_episodes=100, device=device
            )

            estimate_value = eval(environment, partial(target_policy, policy_dqn=policy_dqn), treatment_net, 
                                  num_init_samples=1000,device=device)

            print(truth_value, truth_value_ste)
            print(estimate_value)
            

100%|██████████| 1000/1000 [00:00<00:00, 2837.12it/s]


89.78299846692262 2.2849791419654695
89.08661341094971


100%|██████████| 1000/1000 [00:00<00:00, 3195.41it/s]

89.17712927902214 2.336299925968271
89.07414571380615
